In [1]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import math 

PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'sarah-bucket' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

SEQ_LEN = 50

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Define the project, Bucket and Region and set them.

In [2]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [3]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [4]:
! pwd
! ls

/content/datalab/energy_forcasing/Sarah
ARIMA.csv		 CNN_W2P_model	  sarah_data.csv
BIG_q.ipynb		 data		  test.json
build_local_model.ipynb  Energy_ts	  trained
Build_model_task.ipynb	 hyperparam.yaml  train_model_local.ipynb


In [5]:
## move the csv file to the bucket ~ you rly dont need to do this
## !gsutil cp -p /content/datalab/energy_forcasing/Sarah/sarah_data.csv gs://sarah-bucket/sarah_data.csv

In [6]:
CSV_COLUMNS = ['prediction_date', 'wind_speed_100m', 
  'wind_direction_100m', 
  'temperature', 
  'air_density',
  'pressure', 'precipitation', 'wind_gust', 'radiation', 
  'wind_speed', 'wind_direction', 'price']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df5 = pd.read_csv('/content/datalab/energy_forcasing/Sarah/sarah_data.csv', header = None, names = CSV_COLUMNS)
df5.describe()

,wind_speed_100m,wind_direction_100m,temperature,air_density,pressure,precipitation,wind_gust,radiation,wind_speed,wind_direction,price
count,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000
mean,4.799901,189.582975,14.920355,1.132338,935.286695,0.117364,4.872509,225.250182,3.220986,189.082629,50.153506
std,1.964154,58.126639,9.626527,0.039333,6.105083,0.245561,2.428587,266.299815,1.319704,56.712796,12.448392
min,1.372222,47.883333,-6.044444,1.036111,913.800000,0.000000,1.194444,0.000000,0.855556,50.538889,4.000000
25%,3.534722,142.598611,7.540278,1.106111,932.034722,0.000000,3.284722,0.000000,2.294444,143.750000,43.000000
50%,4.477778,191.908333,14.213889,1.131667,934.761111,0.000000,4.327778,95.000000,2.961111,191.208333,50.890000
75%,5.544444,232.043056,21.062500,1.160556,937.805556,0.100000,5.627778,416.422222,3.850000,230.865278,59.950000
max,15.044444,329.544444,41.044444,1.229444,961.238889,1.700000,16.705556,954.288889,9.738889,327.711111,85.050000


In [7]:
df5.shape[0]

6646

In [8]:
#for ii in range(10):
#   N = 7
#  SEQ = df5.price[(0+N*ii):(N+N*ii)]
np.random.seed(seed=None)
# np.random.uniform(0,6646-N)
6646-50

6596

In [32]:
def to_csv(filename, start, N, J):
  with open(filename, 'w') as ofp:
    for ii in xrange(start, N):
      # nn = int(round(np.random.uniform(low = 0, high = 6466-J),0))
      seq = (df5.price[(ii):(ii+J)]/60) ## trade nn for ii to make an in seq dataset instead of random
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')
      #print(len(seq))
      if len(seq) != J:
        break

import os
try:
  os.makedirs('data/CNN_W2P_model/')
except OSError:
  pass
to_csv('data/CNN_W2P_model/train-1.csv', start = 0, N = 3957, J = 50)  # 3958 sequences
to_csv('data/CNN_W2P_model/valid-1.csv', start = 3958, N = 6596, J = 50) # 2638 eval seq, 40% of the data (6646 - 50)

In [10]:
df2 = pd.read_csv('data/CNN_W2P_model/train-1.csv', header = None)
df2.head()
#df2.shape

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.948333,0.942500,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,...,0.658333,0.500000,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667
1,0.942500,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,...,0.500000,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333
2,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,...,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500
3,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,0.834500,...,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500,1.124667
4,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,0.834500,0.816667,...,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500,1.124667,0.492167


In [11]:
df1 = pd.read_csv('data/CNN_W2P_model/valid-1.csv', header = None) #, names = CSV_COLUMNS)
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.893333,0.550000,0.886500,1.018167,0.668667,0.863333,0.718333,0.960000,0.711500,0.805833,...,0.725000,0.983167,0.946167,0.919833,1.011500,1.116833,1.019833,0.920833,0.794833,1.083333
1,0.550000,0.886500,1.018167,0.668667,0.863333,0.718333,0.960000,0.711500,0.805833,0.766000,...,0.983167,0.946167,0.919833,1.011500,1.116833,1.019833,0.920833,0.794833,1.083333,0.744167
2,0.886500,1.018167,0.668667,0.863333,0.718333,0.960000,0.711500,0.805833,0.766000,1.000000,...,0.946167,0.919833,1.011500,1.116833,1.019833,0.920833,0.794833,1.083333,0.744167,1.001667
3,1.018167,0.668667,0.863333,0.718333,0.960000,0.711500,0.805833,0.766000,1.000000,0.983333,...,0.919833,1.011500,1.116833,1.019833,0.920833,0.794833,1.083333,0.744167,1.001667,0.993333
4,0.668667,0.863333,0.718333,0.960000,0.711500,0.805833,0.766000,1.000000,0.983333,0.970833,...,1.011500,1.116833,1.019833,0.920833,0.794833,1.083333,0.744167,1.001667,0.993333,0.865000


## Fun short cuts I learned while using this notebook:
Cltr + / = comment and un-comment <br>
Shift + tab = un-indent <br>


In [45]:
# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            # row is a string tensor containing the contents of one row
            features = tf.decode_csv(row, record_defaults=DEFAULTS)  # string tensor -> list of 50 rank 0 float tensors
            label = features.pop()  # remove last feature and use as label
            features = tf.stack(features)  # list of rank 0 tensors -> single rank 1 tensor
            return {TIMESERIES_COL: features}, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = dataset.map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely
            dataset = dataset.shuffle(buffer_size=10 * batch_size)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn

N_OUTPUTS = 1
TIMESERIES_COL = 'price'
SEQ_LEN = 50
N_INPUTS = int(SEQ_LEN - N_OUTPUTS)

def init(hparams):
    global SEQ_LEN, DEFAULTS, N_INPUTS
    SEQ_LEN = hparams['sequence_length']
    DEFAULTS = [[0.0] for x in range(0, SEQ_LEN)]
    N_INPUTS = int(SEQ_LEN - N_OUTPUTS)
    
import tensorflow as tf
tf.reset_default_graph()
## make the activtion the same for each run - so results can be compared when testing code
tf.set_random_seed(seed=5)  
features = tf.placeholder(tf.float32, shape=[512, N_INPUTS])

## Testing the linear model
# X = features
# predictions = tf.layers.dense(X, 1, activation=None)

## Testing the DNN model
# X = features
# h1 = tf.layers.dense(inputs=X, units=20, activation=tf.nn.relu6)
# h2 = tf.layers.dense(inputs=h1, units=10, activation=tf.nn.relu6)
# predictions = tf.layers.dense(h2, 1, activation=None)  # linear output: regression

## Testing the CNN
X = x = tf.reshape(features, [-1, N_INPUTS, 1])  # as a 1D "sequence" with only one time-series observation (height)
c1 = tf.layers.conv1d(X, filters=N_INPUTS // 3,
                      kernel_size=3, strides=1,
                      padding='same', activation=tf.nn.relu)
p1 = tf.layers.max_pooling1d(c1, pool_size=2, strides=2)

#     outlen = p1.shape[1] * p1.shape[2]
#     c1flat = tf.reshape(p1, [-1, outlen])
#     h1 = tf.layers.dense(c1flat, 3, activation=tf.nn.relu)
#     predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression
#     return predictions

c2 = tf.layers.conv1d(p1, filters=N_INPUTS // 3,
                      kernel_size=3, strides=1,
                      padding='same', activation=tf.nn.relu)
p2 = tf.layers.max_pooling1d(c2, pool_size=2, strides=2)

outlen = p2.shape[1] * p2.shape[2]
c2flat = tf.reshape(p2, [-1, outlen])
h1 = tf.layers.dense(c2flat, 3, activation=tf.nn.relu)
predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression

# ## Testing RNN shapes.
# x = tf.reshape(features, [-1, N_INPUTS, 1])  ## dim is batch, seq len, 

# # 2. configure the RNN
# cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 3)
# cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 3)
# cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
# outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
# # 'state' is now a tuple containing the final state of each cell layer
# # we use state[1] below to extract the final state of the final layer
# outputs = outputs[:, (N_INPUTS-1):, :] # last one only

# # 3. pass rnn output through a dense layer
# h1 = tf.layers.dense(state[1], cells.output_size // 3, activation=tf.nn.relu)
# predictions = tf.layers.dense(h1, 1, activation=None)  # (?, 1)

init({'sequence_length': 50})
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init) ### initalize the session
    
    features1, labels= sess.run(read_dataset('data/CNN_W2P_model/train-1.csv', None)())
    print("the read in data",features.shape, features1['price'])
    
#     xx = sess.run(x, feed_dict={features: features1['price']} )
#     print("re-formatted data",xx.shape, xx)
    
#     outputs = sess.run(outputs, feed_dict={features: features1['price']})
#     print("the output", outputs.shape, outputs)
    
    preds = sess.run(predictions, feed_dict={features: features1['price']})
    print("the predictions",preds.shape, preds)


('the read in data', TensorShape([Dimension(512), Dimension(49)]), array([[0.9483333 , 0.9425    , 0.9123333 , ..., 0.78466666, 0.75166667,
        0.24733333],
       [0.9425    , 0.9123333 , 0.5       , ..., 0.75166667, 0.24733333,
        1.0016667 ],
       [0.9123333 , 0.5       , 0.781     , ..., 0.24733333, 1.0016667 ,
        1.0083333 ],
       ...,
       [0.5581667 , 0.7353333 , 0.6271667 , ..., 1.0705    , 0.72083336,
        0.8183333 ],
       [0.7353333 , 0.6271667 , 0.928     , ..., 0.72083336, 0.8183333 ,
        0.9115    ],
       [0.6271667 , 0.928     , 1.1166667 , ..., 0.8183333 , 0.9115    ,
        0.7683333 ]], dtype=float32))
('the predictions', (512, 1), array([[0.31186774],
       [0.28489688],
       [0.26647034],
       [0.2531394 ],
       [0.3212092 ],
       [0.2699001 ],
       [0.32881135],
       [0.28834128],
       [0.18310119],
       [0.30654052],
       [0.24526238],
       [0.34367824],
       [0.33231077],
       [0.32137573],
       [0.311280